<a href="https://colab.research.google.com/github/ga642381/ML2021-Spring/blob/main/HW15/HW15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a name="top"></a>
# **HW15 Meta Learning: Few-shot Classification**

Please mail to ntu-ml-2021spring-ta@googlegroups.com if you have any questions.

Useful Links:
1. [Go to hyperparameter setting.](#hyp)
1. [Go to meta algorithm setting.](#modelsetting)
1. [Go to main loop.](#mainloop)

## **Step 0: Check GPU**

In [ ]:
!nvidia-smi

In [ ]:
#@markdown ### Install `qqdm`
# Check if installed
try:
    import qqdm
except:
    ! pip install qqdm > /dev/null 2>&1
print("Done!")

Done!


## **Step 1: Download Data**

Run the cell to download data, which has been pre-processed by TAs.  
The dataset has been augmented, so extra data augmentation is not required.


In [ ]:
workspace_dir = '.'

# gdown is a package that downloads files from google drive
!gdown --id 1FLDrQ0k-iJ-mk8ors0WItqvwgu0w9J0U \
    --output "{workspace_dir}/Omniglot.tar.gz"

Downloading...
From: https://drive.google.com/uc?id=1FLDrQ0k-iJ-mk8ors0WItqvwgu0w9J0U
To: /content/Omniglot.tar.gz
26.4MB [00:00, 124MB/s] 


### Decompress the dataset

Since the dataset is quite large, please wait and observe the main program [here](#mainprogram).  
You can come back here later by [*back to pre-process*](#preprocess).

In [ ]:
# Use `tar' command to decompress
!tar -zxf "{workspace_dir}/Omniglot.tar.gz"          \
    -C "{workspace_dir}/"

### Data Preview

Just look at some data in the dataset.

In [ ]:
from PIL import Image
from IPython.display import display
for i in range(10, 20):
    im = Image.open("Omniglot/images_background/Japanese_(hiragana).0/character13/0500_" + str (i) + ".png")
    display(im)

## **Step 2: Build the model**

### Library importation

In [ ]:
# Import modules we need
import glob
from collections import OrderedDict

import numpy as np

try:
    from qqdm.notebook import qqdm as tqdm
except ModuleNotFoundError:
    from tqdm.auto import tqdm

import torch, torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms

from PIL import Image
from IPython.display import display

device = "cuda" if torch.cuda.is_available() else "cpu"

# fix random seeds
random_seed = 0
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(random_seed)

### Model Construction Preliminaries

Since our task is image classification, we need to build a CNN-based model.  
However, to implement MAML algorithm, we should adjust some code in `nn.Module`.


Take a look at MAML pseudocode...

<img src="https://i.imgur.com/9aHlvfX.png" width="50%" />

On the 10-th line, what we take gradients on are those $\theta$ representing  
<font color="#0CC">**the original model parameters**</font> (outer loop) instead of those in  the  
<font color="#0C0">**inner loop**</font>, so we need to use `functional_forward` to compute the output  
logits of input image instead of `forward` in `nn.Module`.

The following defines these functions.

<!-- 由於在第10行，我們是要對原本的參數 θ 微分，並非 inner-loop (Line5~8) 的 θ' 微分，因此在 inner-loop，我們需要用 functional forward 的方式算出 input image 的 output logits，而不是直接用 nn.module 裡面的 forward（直接對 θ 微分）。在下面我們分別定義了 functional forward 以及 forward 函數。 -->

### Model block definition

In [ ]:
def ConvBlock(in_ch: int, out_ch: int):
    return nn.Sequential(
        nn.Conv2d(in_ch, out_ch, 3, padding=1),
        nn.BatchNorm2d(out_ch),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )

def ConvBlockFunction(x, w, b, w_bn, b_bn):
    x = F.conv2d(x, w, b, padding=1)
    x = F.batch_norm(x,
                     running_mean=None,
                     running_var=None,
                     weight=w_bn, bias=b_bn,
                     training=True)
    x = F.relu(x)
    x = F.max_pool2d(x, kernel_size=2, stride=2)
    return x

### Model definition

In [ ]:
class Classifier(nn.Module):
    def __init__(self, in_ch, k_way):
        super(Classifier, self).__init__()
        self.conv1 = ConvBlock(in_ch, 64)
        self.conv2 = ConvBlock(64, 64)
        self.conv3 = ConvBlock(64, 64)
        self.conv4 = ConvBlock(64, 64)
        self.logits = nn.Linear(64, k_way)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = x.view(x.shape[0], -1)
        x = self.logits(x)
        return x

    def functional_forward(self, x, params):
        '''
        Arguments:
        x: input images [batch, 1, 28, 28]
        params: model parameters, 
                i.e. weights and biases of convolution
                     and weights and biases of 
                                   batch normalization
                type is an OrderedDict

        Arguments:
        x: input images [batch, 1, 28, 28]
        params: 模型的參數，也就是 convolution 的 weight
                跟 bias，以及 batch normalization 的
                weight 跟 bias
                這是一個 OrderedDict
        '''
        for block in [1, 2, 3, 4]:
            x = ConvBlockFunction(
                x,
                params[f'conv{block}.0.weight'],
                params[f'conv{block}.0.bias'],
                params.get(f'conv{block}.1.weight'),
                params.get(f'conv{block}.1.bias'))
        x = x.view(x.shape[0], -1)
        x = F.linear(x,
                     params['logits.weight'],
                     params['logits.bias'])
        return x

### Create Label

This function is used to create labels.  
In a N-way K-shot few-shot classification problem,
each task has `n_way` classes, while there are `k_shot` images for each class.  
This is a function that creates such labels.


In [ ]:
def create_label(n_way, k_shot):
    return (torch.arange(n_way)
                 .repeat_interleave(k_shot)
                 .long())

# Try to create labels for 5-way 2-shot setting
create_label(5, 2)

tensor([0, 0, 1, 1, 2, 2, 3, 3, 4, 4])

### Accuracy calculation

In [ ]:
def calculate_accuracy(logits, val_label):
    """ utility function for accuracy calculation """
    acc = np.asarray([(
        torch.argmax(logits, -1).cpu().numpy() == val_label.cpu().numpy())]
        ).mean() 
    return acc

### Define Dataset

Define the dataset.  
The dataset returns images of a random character, with (`k_shot + q_query`) images,  
so the size of returned tensor is `[k_shot+q_query, 1, 28, 28]`.  


In [ ]:
class Omniglot(Dataset):
    def __init__(self, data_dir, k_way, q_query):
        self.file_list = [f for f in glob.glob(
            data_dir + "**/character*", 
            recursive=True)]
        self.transform = transforms.Compose(
                            [transforms.ToTensor()])
        self.n = k_way + q_query

    def __getitem__(self, idx):
        sample = np.arange(20)

        # For random sampling the characters we want.
        np.random.shuffle(sample) 
        img_path = self.file_list[idx]
        img_list = [f for f in glob.glob(
            img_path + "**/*.png", recursive=True)]
        img_list.sort()
        imgs = [self.transform(
            Image.open(img_file)) 
            for img_file in img_list]
        # `k_way + q_query` examples for each character
        imgs = torch.stack(imgs)[sample[:self.n]] 
        return imgs

    def __len__(self):
        return len(self.file_list)

## **Step 3: Core MAML**

Here is the main Meta Learning algorithm.  
The algorithm is exactly the same as the paper.  
What the function does is to update the parameters using "the data of a meta-batch."
Here we implement the second-order MAML (inner_train_step = 1), according to [the slides of meta learning in 2019 (p. 13 ~ p.18)](http://speech.ee.ntu.edu.tw/~tlkagk/courses/ML_2019/Lecture/Meta1%20(v6).pdf#page=13&view=FitW)

As for the mathematical derivation of the first-order version, please refer to [p.25 of the slides in 2019](http://speech.ee.ntu.edu.tw/~tlkagk/courses/ML_2019/Lecture/Meta1%20(v6).pdf#page=25&view=FitW).

The following is the algorithm with some explanation.

In [ ]:
def OriginalMAML(
    model, optimizer, x, n_way, k_shot, q_query, loss_fn,
    inner_train_step=1, inner_lr=0.4, train=True):
    criterion, task_loss, task_acc = loss_fn, [], []

    for meta_batch in x:
        # Get data
        support_set = meta_batch[: n_way * k_shot]  
        query_set = meta_batch[n_way * k_shot :]    
        
        # Copy the params for inner loop
        fast_weights = OrderedDict(model.named_parameters())
        
        ### ---------- INNER TRAIN LOOP ---------- ###
        for inner_step in range(inner_train_step): 
            # Simply training
            train_label = create_label(n_way, k_shot)      \
                          .to(device)
            logits = model.functional_forward(
                                  support_set, fast_weights)
            loss = criterion(logits, train_label)
            # Inner gradients update! vvvvvvvvvvvvvvvvvvvv #
            """ Inner Loop Update """                      #
            grads = torch.autograd.grad(                   #
                loss, fast_weights.values(),               #
                create_graph=True)                         #
            # Perform SGD                                  #
            fast_weights = OrderedDict(                    #
                (name, param - inner_lr * grad)            #
                for ((name, param), grad)                  #
                    in zip(fast_weights.items(), grads))   #
            # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ #

        ### ---------- INNER VALID LOOP ---------- ###
        val_label = create_label(n_way, q_query).to(device)
                                
        # Collect gradients for outer loop
        logits = model.functional_forward(
                                  query_set, fast_weights) 
        loss = criterion(logits, val_label)
        task_loss.append(loss)
        task_acc.append(
                calculate_accuracy(logits, val_label))

    # Update outer loop
    model.train()
    optimizer.zero_grad()

    meta_batch_loss = torch.stack(task_loss).mean()
    if train:
        meta_batch_loss.backward()  # <--- May change later!
        optimizer.step()
    task_acc = np.mean(task_acc)
    return meta_batch_loss, task_acc

## Variations of MAML

### First-order approximation of MAML (FOMAML)

Slightly modify the MAML mentioned earlier, applying first-order approximation to decrease amount of computation.

### Almost No Inner Loop (ANIL)

The algorithm from [this paper](https://arxiv.org/abs/1909.09157), using the technique of feature reuse to decrease amount of computation.

To finish the modification required, we need to change some blocks of the MAML algorithm.  
Below, we have replace three parts that may be modified as functions.  
Please choose to replace the functions with their alternative versions to complete the algorithm.

### Part 1: Inner loop update

MAML

In [ ]:
def inner_update_MAML(fast_weights, loss, inner_lr):
    """ Inner Loop Update """
    grads = torch.autograd.grad(
        loss, fast_weights.values(), create_graph=True)
    # Perform SGD
    fast_weights = OrderedDict(
        (name, param - inner_lr * grad)
        for ((name, param), grad) in zip(fast_weights.items(), grads))
    return fast_weights

Alternatives

In [ ]:
def inner_update_alt1(fast_weights, loss, inner_lr):
    grads = torch.autograd.grad(
        loss, fast_weights.values(), create_graph=False)
    # Perform SGD
    fast_weights = OrderedDict(
        (name, param - inner_lr * grad)
        for ((name, param), grad) in zip(fast_weights.items(), grads))
    return fast_weights

def inner_update_alt2(fast_weights, loss, inner_lr):
    grads = torch.autograd.grad(
        loss, list(fast_weights.values())[-2:], create_graph=True)
    # Split out the logits
    for ((name, param), grad) in zip(
        list(fast_weights.items())[-2:], grads):
        fast_weights[name] = param - inner_lr * grad
    return fast_weights

### Part 2: Collect gradients

MAML  
(Actually do nothing as gradients are computed by PyTorch automatically.)

In [ ]:
def collect_gradients_MAML(
    special_grad: OrderedDict, fast_weights, model, len_data):
    """ Actually do nothing (just backwards later) """
    return special_grad

Alternatives

In [ ]:
def collect_gradients_alt(
    special_grad: OrderedDict, fast_weights, model, len_data):
    """ Special gradient calculation """
    diff = OrderedDict(
        (name, params - fast_weights[name]) 
        for (name, params) in model.named_parameters())
    for name in diff:
        special_grad[name] = special_grad.get(name, 0) + diff[name] / len_data
    return special_grad

### Part 3: Outer loop gradients calculation

MAML  
(Simply call PyTorch `backward`.)

In [ ]:
def outer_update_MAML(model, meta_batch_loss, grad_tensors):
    """ Simply backwards """
    meta_batch_loss.backward()

Alternatives

In [ ]:
def outer_update_alt(model, meta_batch_loss, grad_tensors):
    """ Replace the gradients
                        with precalculated tensors """
    for (name, params) in model.named_parameters():
        params.grad = grad_tensors[name]

### Complete the algorithm
Here we have wrapped the algorithm in `MetaAlgorithmGenerator`.  
You can get your modified algorithm by filling in like this:
```python
MyAlgorithm = MetaAlgorithmGenerator(inner_update=inner_update_alt2)
```
Default the three blocks will be filled with that of `MAML`.

In [ ]:
def MetaAlgorithmGenerator(
    inner_update      = inner_update_MAML, 
    collect_gradients = collect_gradients_MAML, 
    outer_update      = outer_update_MAML):

    global calculate_accuracy

    def MetaAlgorithm(
        model, optimizer, x, n_way, k_shot, q_query, loss_fn,
        inner_train_step=1, inner_lr=0.4, train=True): 
        criterion = loss_fn
        task_loss, task_acc = [], []
        special_grad = OrderedDict()  # Added for variants!

        for meta_batch in x:
            support_set = meta_batch[: n_way * k_shot]  
            query_set = meta_batch[n_way * k_shot :]    
            
            fast_weights = OrderedDict(model.named_parameters())
            
            ### ---------- INNER TRAIN LOOP ---------- ###
            for inner_step in range(inner_train_step): 
                train_label = create_label(n_way, k_shot).to(device)
                logits = model.functional_forward(support_set, fast_weights)
                loss = criterion(logits, train_label)

                fast_weights = inner_update(fast_weights, loss, inner_lr)

            ### ---------- INNER VALID LOOP ---------- ###
            val_label = create_label(n_way, q_query).to(device)
            # FIXME: W for val?
            special_grad = collect_gradients(
                special_grad, fast_weights, model, len(x))
                                    
            # Collect gradients for outer loop
            logits = model.functional_forward(query_set, fast_weights) 
            loss = criterion(logits, val_label)
            task_loss.append(loss)
            task_acc.append(calculate_accuracy(logits, val_label))

        # Update outer loop
        model.train()
        optimizer.zero_grad()

        meta_batch_loss = torch.stack(task_loss).mean()
        if train:
            # Notice the update part!
            outer_update(model, meta_batch_loss, special_grad)
            optimizer.step()
        task_acc = np.mean(task_acc)
        return meta_batch_loss, task_acc
    return MetaAlgorithm

In [ ]:
#@title Here is the answer hidden, please fill in yourself!
Give_me_the_answer = True #@param {"type": "boolean"}

def HiddenAnswer():
    MAML    = MetaAlgorithmGenerator()
    FOMAML  = MetaAlgorithmGenerator(inner_update=inner_update_alt1)
    ANIL    = MetaAlgorithmGenerator(inner_update=inner_update_alt2)
    return MAML, FOMAML, ANIL

In [ ]:
# `HiddenAnswer` is hidden in the last cell.
if Give_me_the_answer:
    MAML, FOMAML, ANIL = HiddenAnswer()
else:  
    # TODO: Please fill in the function names \
    # as the function arguments to finish the algorithm.
    MAML    = MetaAlgorithmGenerator()
    FOMAML  = MetaAlgorithmGenerator()
    ANIL    = MetaAlgorithmGenerator()

## **Step 4: Initialization**

After defining all components we need, the following initialize a model before training.

<a name="hyp"></a>
### Hyperparameters 
[Go back to top!](#top)

In [ ]:
n_way = 5
k_shot = 1
q_query = 1
inner_train_step = 1
inner_lr = 0.4
meta_lr = 0.001
meta_batch_size = 32
max_epoch = 30
eval_batches = test_batches = 20
train_data_path = './Omniglot/images_background/'
test_data_path = './Omniglot/images_evaluation/'    

### Dataloader initialization

In [ ]:
def dataloader_init(datasets, num_workers=2):
    train_set, val_set, test_set = datasets
    train_loader = DataLoader(train_set,
                            # The "batch_size" here is not \
                            #    the meta batch size, but  \
                            #    how many different        \
                            #    characters in a task,     \
                            #    i.e. the "n_way" in       \
                            #    few-shot classification.
                            batch_size=n_way,
                            num_workers=num_workers,
                            shuffle=True,
                            drop_last=True)
    val_loader = DataLoader(val_set,
                            batch_size=n_way,
                            num_workers=num_workers,
                            shuffle=True,
                            drop_last=True)
    test_loader = DataLoader(test_set,
                            batch_size=n_way,
                            num_workers=num_workers,
                            shuffle=True,
                            drop_last=True)
    train_iter = iter(train_loader)
    val_iter = iter(val_loader)
    test_iter = iter(test_loader)
    return (train_loader, val_loader, test_loader), \
           (train_iter, val_iter, test_iter)

train_set, val_set = torch.utils.data.random_split(
    Omniglot(train_data_path, k_shot, q_query), [3200, 656])
test_set = Omniglot(test_data_path, k_shot, q_query)

(train_loader, val_loader, test_loader), \
(train_iter, val_iter, test_iter) = dataloader_init(
                             (train_set, val_set, test_set))

### Model & optimizer initialization

In [ ]:
def model_init():
    meta_model = Classifier(1, n_way).to(device)
    optimizer = torch.optim.Adam(meta_model.parameters(), 
                                 lr=meta_lr)
    loss_fn = nn.CrossEntropyLoss().to(device)
    return meta_model, optimizer, loss_fn

meta_model, optimizer, loss_fn = model_init()

### Utility function to get a meta-batch

In [ ]:
def get_meta_batch(meta_batch_size,
                   k_shot, q_query, 
                   data_loader, iterator):
    data = []
    for _ in range(meta_batch_size):
        try:
            # a "task_data" tensor is representing \
            #     the data of a task, with size of \
            #     [n_way, k_shot+q_query, 1, 28, 28]
            task_data = iterator.next()  
        except StopIteration:
            iterator = iter(data_loader)
            task_data = iterator.next()
        train_data = (task_data[:, :k_shot]
                      .reshape(-1, 1, 28, 28))
        val_data = (task_data[:, k_shot:]
                    .reshape(-1, 1, 28, 28))
        task_data = torch.cat(
            (train_data, val_data), 0)
        data.append(task_data)
    return torch.stack(data).to(device), iterator

<a name="modelsetting"></a>
### Choose the meta learning algorithm
[Go back to top!](#top)

In [ ]:
# You can change this to `FOMAML` or `ANIL`
MetaAlgorithm = MAML

<a name="mainprog" id="mainprog"></a>
## **Step 5: Main program for training & testing**

### Start training!
<a name="mainloop"></a>
[Go back to top!](#top)

In [ ]:
for epoch in range(max_epoch):
    print("Epoch %d" % (epoch + 1))
    train_meta_loss = []
    train_acc = []
    # The "step" here is a meta-gradinet update step
    for step in tqdm(range(
            len(train_loader) // meta_batch_size)): 
        x, train_iter = get_meta_batch(
            meta_batch_size, k_shot, q_query, 
            train_loader, train_iter)
        meta_loss, acc = MetaAlgorithm(
            meta_model, optimizer, x, 
            n_way, k_shot, q_query, loss_fn)
        train_meta_loss.append(meta_loss.item())
        train_acc.append(acc)
    print("  Loss    : ", "%.3f" % (np.mean(train_meta_loss)), end='\t')
    print("  Accuracy: ", "%.3f %%" % (np.mean(train_acc) * 100))

    # See the validation accuracy after each epoch.
    # Early stopping is welcomed to implement.
    val_acc = []
    for eval_step in tqdm(range(
            len(val_loader) // (eval_batches))):
        x, val_iter = get_meta_batch(
            eval_batches, k_shot, q_query, 
            val_loader, val_iter)
        # We update three inner steps when testing.
        _, acc = MetaAlgorithm(meta_model, optimizer, x, 
                      n_way, k_shot, q_query, 
                      loss_fn, 
                      inner_train_step=3, 
                      train=False) 
        val_acc.append(acc)
    print("  Validation accuracy: ", "%.3f %%" % (np.mean(val_acc) * 100))

Epoch 1


 Iters    Elapsed Time      Speed                                               
 20/20  00:00:11<00:00:00  1.71it/s                                             

IpythonBar(children=(HTML(value='  0.0%'), FloatProgress(value=0.0)))

  Loss    :  1.961	  Accuracy:  29.813 %


 Iters    Elapsed Time      Speed                                               
  6/6   00:00:02<00:00:00  2.23it/s                                             

...

IpythonBar(children=(HTML(value='  0.0%'), FloatProgress(value=0.0)))

  Loss    :  1.662	  Accuracy:  22.969 %


 Iters    Elapsed Time      Speed                                               
  6/6   00:00:02<00:00:00  2.12it/s                                             

IpythonBar(children=(HTML(value='  0.0%'), FloatProgress(value=0.0)))

  Validation accuracy:  23.500 %


### Testing the result

In [ ]:
test_acc = []
for test_step in tqdm(range(
        len(test_loader) // (test_batches))):
    x, test_iter = get_meta_batch(
        test_batches, k_shot, q_query, 
        test_loader, test_iter)
    # testing 時，我們更新三次 inner-step
    _, acc = MetaAlgorithm(meta_model, optimizer, x, 
                  n_way, k_shot, q_query, loss_fn, 
                  inner_train_step=3, train=False)
    test_acc.append(acc)
print("  Testing accuracy: ", "%.3f %%" % (np.mean(test_acc) * 100))

 Iters    Elapsed Time      Speed                                               
  6/6   00:00:02<00:00:00  2.10it/s                                             

IpythonBar(children=(HTML(value='  0.0%'), FloatProgress(value=0.0)))

  Testing accuracy:  22.167 %


## **Reference**
1. Chelsea Finn, Pieter Abbeel, & Sergey Levine. (2017). [Model-Agnostic Meta-Learning for Fast Adaptation of Deep Networks.](https://arxiv.org/abs/1909.09157)
1. Aniruddh Raghu, Maithra Raghu, Samy Bengio, & Oriol Vinyals. (2020). [Rapid Learning or Feature Reuse? Towards Understanding the Effectiveness of MAML.](https://arxiv.org/abs/1909.09157)